In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
        .appName('Prediction_Attrition_Client')
        .master('local[*]')
        .getOrCreate())

In [2]:
print(spark.version)

4.0.1
